Adapted from https://towardsdatascience.com/backtest-trading-strategies-with-pandas-vectorized-backtesting-26001b0ba3a5

Also see https://github.com/gylx/Financial-Machine-Learning-Articles/

In [22]:
import numpy as np
import pandas as pd
import os
import sys
import glob
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import datetime as datetime
import time

In [23]:
file_path = r'C:\Users\torku\Documents\crypto_data'
files = glob.glob(file_path + r'\*.csv')

In [24]:
df = pd.read_csv(files[0])
idx = pd.to_datetime(df['timestamp'])
df = df.set_index(idx)
df = df.drop(['timestamp'], axis = 1)
df.head()

,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
timestamp,,,,,,,,,,,
2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,1.775183,1502942459999,7564.906851,3,0.075183,320.390851,7960.540180
2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.000000,1502942519999,0.000000,0,0.000000,0.000000,7960.452630
2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.261074,1502942579999,1117.542921,2,0.261074,1117.542921,7960.375295
2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.012008,1502942639999,51.171852,3,0.012008,51.171852,7960.375295
2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.140796,1502942699999,599.999338,1,0.140796,599.999338,7960.375295


In [25]:
df_sub = df[['close']]
df_sub = df_sub.loc[df_sub.index > pd.Timestamp('2021-1-1 01:00:00')]

#price for single coin
prices = df_sub['close']

#log difference for prices along all time steps
rs = prices.apply(np.log).diff(1).fillna(0)

#set moving average windows  1440 min per day
w1 = 150 #2 * 1440 # short-term moving average window
w2 = 6500 #14 * 1440 # long-term moving average window

x = df_sub.index
ma_x = prices.rolling(w1).mean() - prices.rolling(w2).mean()

In [26]:
#display positions
pos = ma_x.apply(np.sign) #give -1 0 1 depending on sign
pos = pos.apply(lambda x: 0 if x < 1 else x) #not short selling
#shift +1 to avoid look ahead bias
my_rs = pos.shift(1) * rs #doesn't include costs
my_rs_cumsum = my_rs.cumsum().apply(np.exp)

tc_perc = 0.001 #Binance transaction cost of 0.1
#at every point where we're trading, want to mark the transaction
#this should be the portfolio balance * transaction cost percent
delta_pos = pos.diff(1).abs()
my_tcs = tc_perc * delta_pos # compute transaction costs
my_rs_w_cost = (pos.shift(1) * rs) - my_tcs
my_rs_w_cost_cumsum = np.subtract(my_rs, my_tcs).cumsum().apply(np.exp)

pos_baseline = abs(pos)
base_rs = pos_baseline.shift(1) * rs
base_rs_cumsum = base_rs.cumsum().apply(np.exp)

In [27]:
fig = make_subplots(rows = 4, cols = 1, shared_xaxes=True,
                    vertical_spacing = 0.02)
thinner = 735

fig.add_scatter(x = x[::thinner], y = prices[::thinner], mode = 'lines',
    name = 'actual', row = 1, col = 1)

fig.add_scatter(x = x[::thinner], y = prices.rolling(w1).mean()[::thinner], mode = 'lines',
    name = 'ma_short', row = 1, col = 1)

fig.add_scatter(x = x[::thinner], y = prices.rolling(w2).mean()[::thinner], mode = 'lines',
    name = 'ma long',row = 1, col = 1)

fig.add_scatter(x = x[::thinner], y = ma_x[::thinner], mode = 'lines',
    name = 'moving average difference', row = 2, col = 1)

fig.add_scatter(x = x[::thinner], y = pos[::thinner], mode = 'lines',
    name = 'position', row = 3, col = 1)

fig.add_scatter(x = x[::thinner], y = my_rs_cumsum[::thinner], mode = 'lines',
    name = 'my_rs_cumsum', row = 4, col = 1)

fig.add_scatter(x = x[::thinner], y = my_rs_w_cost_cumsum[::thinner], mode = 'lines',
    name = 'my_rs_cumsum_with_fee', row = 4, col = 1)

fig.add_scatter(x = x[::thinner], y = base_rs_cumsum[::thinner], mode = 'lines',
    name = 'base_rs_cumsum', row = 4, col = 1)

In [ ]:
set_of_w_rs

In [13]:
df_sub = df[['close']]
df_sub = df_sub.loc[df_sub.index > pd.Timestamp('2021-1-1 01:00:00')]

#price for single coin
prices = df_sub['close']

#log difference for prices along all time steps
rs = prices.apply(np.log).diff(1).fillna(0)
w1_list = [x * 15 for x in range(100)]
w1_list.pop(0)


w2_list = [120 * x for x in range(100)]
w2_list.pop(0)

tc_perc = 0.001

x = df_sub.index


set_of_w1 = []
set_of_w2 = []
set_of_w_rs = []
st = time.time()
for w1 in w1_list:
    et = time.time()
    print('Started ' + str(w1) + ' at ' + str(et - st))
    
    for w2 in w2_list: 
        ma_x = prices.rolling(w1).mean() - prices.rolling(w2).mean()

        #display positions
        pos = ma_x.apply(np.sign) #give -1 0 1 depending on sign
        pos = pos.apply(lambda x: 0 if x < 1 else x) #not short selling
        
        #shift +1 to avoid look ahead bias
        my_rs = pos.shift(1) * rs #doesn't include costs
        my_rs_cumsum = my_rs.cumsum().apply(np.exp)

        #at every point where we're trading, want to mark the transaction
        #this should be the portfolio balance * transaction cost percent
        delta_pos = pos.diff(1).abs()
        my_tcs = tc_perc * delta_pos # compute transaction costs
        my_rs_w_cost = (pos.shift(1) * rs) - my_tcs
        my_rs_w_cost_cumsum = np.subtract(my_rs, my_tcs).cumsum().apply(np.exp)
        set_of_w1.append(w1)
        set_of_w2.append(w2)
        set_of_w_rs.append(my_rs_w_cost_cumsum[-1])

Started 15 at 0.0
Started 30 at 6.1238062381744385
Started 45 at 12.220729351043701
Started 60 at 18.11049199104309
Started 75 at 24.11885142326355
Started 90 at 30.74315595626831
Started 105 at 37.60601806640625
Started 120 at 44.18316030502319
Started 135 at 50.90066742897034
Started 150 at 57.68947196006775
Started 165 at 64.41526293754578
Started 180 at 71.22385025024414
Started 195 at 78.08545851707458
Started 210 at 84.90834856033325
Started 225 at 91.74330997467041
Started 240 at 98.57913446426392
Started 255 at 105.4438886642456
Started 270 at 112.29297471046448
Started 285 at 119.17858862876892
Started 300 at 126.01828670501709
Started 315 at 132.93591976165771
Started 330 at 139.76510524749756
Started 345 at 146.70269584655762
Started 360 at 153.63612174987793
Started 375 at 160.59608149528503
Started 390 at 167.51311612129211
Started 405 at 174.5274941921234
Started 420 at 181.50816226005554
Started 435 at 188.50304770469666
Started 450 at 195.52378582954407
Started 465 at 2

In [31]:
import plotly.express as px
df = pd.DataFrame(dict(w1 = set_of_w1, w2 = set_of_w2, z = set_of_w_rs))
fig = px.density_heatmap(data_frame = df, x = 'w1', y = 'w2', z = 'z')
fig.show()

In [35]:
df.sort_values(by = 'z', ascending = False)

,w1,w2,z
941,150,6120,2.163916
841,135,6000,2.127778
643,105,6000,2.113975
544,90,6000,2.084609
940,150,6000,2.066871
...,...,...,...
99,30,120,0.237613
1387,225,240,0.214623
792,135,120,0.163681
0,15,120,0.159910
